<a href="https://colab.research.google.com/github/shuuuuuuhei/kaggle-Recod.ai-LUC-Scientific-Image-Forgery-Detection/blob/main/20251026_Recod_ai_LUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Kaggle CLI をインストール
!pip install -q kaggle

# kaggle.json を ~/.kaggle/ にコピー（Drive から）
!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# データをダウンロード
!kaggle competitions download -c recodai-luc-scientific-image-forgery-detection

# 解凍
!unzip -q recodai-luc-scientific-image-forgery-detection.zip -d /content/data

 99% 4.69G/4.75G [01:15<00:04, 16.0MB/s]
100% 4.75G/4.75G [01:16<00:00, 67.1MB/s]
